In [1]:
import sys

import matplotlib.pyplot as plt
## save variables
import pickle
## folder names
from glob import glob
## standard libraries
import numpy as np
## division for train and test
from sklearn.model_selection import train_test_split
##
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize

from keras.activations import softmax
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Flatten, Dropout, Convolution2D, MaxPooling2D, AveragePooling2D
from keras import backend as K
K.set_image_dim_ordering('th')
from keras.optimizers import SGD
from keras.utils import np_utils

import sys
#!{sys.executable} -m pip install hyperas --user
#!{sys.executable} -m pip install networkx==1.11 --user
#!{sys.executable} -m pip install --upgrade keras --user

from hyperas import optim
from hyperas.distributions import choice, uniform, conditional
from hyperopt import Trials, STATUS_OK, tpe

Using TensorFlow backend.


## KCenter Functions

## No-K Functions

In [2]:
def plotConfusionMatrix(predictions, true_labels, labels):
    k = true_labels.shape[1]
    n = true_labels.shape[0]
    confusion_matrix = np.zeros((k,k))

    for l in range(n):
        decision = np.zeros(k)
        j = np.argmax(predictions[l])
        decision[j] = 1
        i = np.argmax(true_labels[l])
        confusion_matrix[i,j] +=1
        
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(confusion_matrix)
    plt.title('Confusion matrix of the classifier')
    fig.colorbar(cax)
    ax.set_xticklabels([''] + labels)
    ax.set_yticklabels([''] + labels)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()
    

def train_test_creator(dic, test_size = 0.2):
    X = []
    Y = []
    if type(dic)==dict:
        length = len(dic)
        for count, key in enumerate(dic):
            tmp = dic[key]
            label = np.array(count)
            label = np.resize(label, (tmp.shape[0],1))
            X.append(tmp)
            Y.append(label)
            #print(key, ' ', count)
    else:
        return -1
    X = np.vstack(X)
    Y = np.vstack(Y)
    
    Y = np_utils.to_categorical(Y, np.max(Y)+1)
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size)
    X_train = np.reshape(X_train, ( X_train.shape[0],1, X_train.shape[1], X_train.shape[2]))
    X_test = np.reshape(X_test, ( X_test.shape[0], 1, X_test.shape[1], X_test.shape[2]))
    with open('variables/train_test_split.pkl', 'wb') as f:  
        pickle.dump(X_train, f)
        pickle.dump(y_train, f)
        pickle.dump(X_test, f)
        pickle.dump(y_test, f)    
    return X_train, y_train, X_test, y_test 

In [3]:
#with open('variables/rawDict.pkl', 'rb') as f:  
    #rawDict = pickle.load(f)
with open('variables/noiseDict.pkl', 'rb') as f:  
    noiseDict = pickle.load(f)    
with open('variables/mfccDict.pkl', 'rb') as f:  
    mfccDict = pickle.load(f)
with open('variables/fbankDict.pkl', 'rb') as f:  
    fbankDict = pickle.load(f)    


In [4]:
coreKey = ["yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go", "zero",
           "one", "two", "three", "four", "five", "six", "seven", "eight", "nine"]

numbers = {'one', 'two', 'three','four','five','six','seven','eight','nine'}
words = {"yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go", "zero"}
# ecco, bravo.
# ahah ma che buggata è sta barra che avanza
#c'è di peggio dai...
#ma sei sicuro che stia funzinoando bene?
#No per niente è il tuo algoritmo ahah
#allora perfetto

used = coreKey

# Scriviamo qui per il dataset k-center

### Estrazione dataset non k-center

In [5]:
x_train, y_train, x_test, y_test = train_test_creator({k: mfccDict[k] for k in mfccDict.keys() & used })

# A' mejo rete

In [ ]:
#def soft()
cnn = Sequential()

cnn.add(Convolution2D(32, (5,4),  strides = (1,1), padding="valid",  activation="softplus",
                      input_shape=(1, x_train.shape[2], x_train.shape[3])))

cnn.add(MaxPooling2D(pool_size=(3,2)))
cnn.add(Dropout(0.25))


cnn.add(Convolution2D(64, (4,2),  strides = (1,1), padding="valid", activation="softplus"))
cnn.add(Convolution2D(128, (4,3),  strides = (1,1), padding="valid", activation="softplus"))

cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))


cnn.add(Flatten())

cnn.add(Dense(80, activation="softplus"))
cnn.add(Dropout(0.6))

#cnn.add(Dense(30, activation="sigmoid"))
#cnn.add(Dropout(0.35))

cnn.add(Dense(y_train.shape[1], activation="softmax"))

# define optimizer and objective, compile cnn
cnn.summary()

In [ ]:
epoch = 40
cnn.compile(loss="categorical_crossentropy", optimizer="adamax", metrics=['accuracy'])
cnn.fit(X_train, y_train, epochs=epoch, validation_data=(x_test, y_test), batch_size=round(x_train.shape[0]/250))

In [ ]:
preds = cnn.predict(x_test)
plotConfusionMatrix(preds,y_test,list(used))
loss, precision = cnn.evaluate(x_test,y_test)
print ("Precision: ", round(precision*100,2),"%")

In [ ]:
import os.path

dest_directory = 'model_backup/'
if not os.path.exists(dest_directory):
      os.makedirs(dest_directory)
name = 'cnn.bak'
cnn.save(dest_directory + name)

#bak = load_model(dest_directory + name)


# Looking for hyperparameters

In [7]:
def data():
    with open('variables/train_test_split.pkl', 'rb') as f: 
        x_train = pickle.load(f)
        y_train = pickle.load(f)
        x_test = pickle.load(f)
        y_test = pickle.load(f) 
    return x_train, y_train, x_test, y_test 

def create_model(x_train, y_train, x_test, y_test):
    cnn = Sequential()
    input_shape = (1, x_train.shape[2], x_train.shape[3])
    cnn.add(Convolution2D({{choice([16,32,64])}}, (5,4),  strides = (1,1), padding="valid", input_shape=input_shape))
    cnn.add(Activation("softplus"))
    cnn.add(MaxPooling2D(pool_size=(3,2)))
    cnn.add(Dropout({{uniform(0, .5)}}))

    cnn.add(Convolution2D({{choice([32,64,128])}}, (4,2),  strides = (1,1), padding="valid"))
    cnn.add(Activation("softplus"))
    cnn.add(Convolution2D({{choice([64,128,256])}}, (4,3),  strides = (1,1), padding="valid"))
    cnn.add(Activation("softplus"))
    cnn.add(MaxPooling2D(pool_size=(2,2)))
    cnn.add(Dropout({{uniform(0, .5)}}))

    cnn.add(Flatten())

    cnn.add(Dense({{choice([40, 60, 80, 100])}}))
    cnn.add(Activation("softplus"))
    cnn.add(Dropout({{uniform(0, 1)}}))
            
    if conditional({{choice(['three', 'four'])}}) == 'four':
        cnn.add(Dense({{choice([20, 30, 40])}}))
        cnn.add(Activation("softplus"))
        cnn.add(Dropout({{uniform(0, .5)}}))

    cnn.add(Dense(y_train.shape[1]))
    cnn.add(Activation({{choice(['softmax', 'sigmoid'])}}))
    cnn.compile(loss="categorical_crossentropy", optimizer="adamax", metrics=['accuracy'])
            
    cnn.fit(x_train, y_train,
              batch_size={{choice([128, 256])}},
              epochs=5,
              verbose=2,
              validation_data=(x_test, y_test))
    score, acc = cnn.evaluate(x_test, y_test, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': cnn}

In [9]:
#from keras.layers.core import Dense, Dropout, Activation
best_run, best_model = optim.minimize(model=create_model, 
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=5,
                                      trials=Trials(),
                                      notebook_name='processing')

>>> Imports:
#coding=utf-8

try:
    import sys
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    import pickle
except:
    pass

try:
    from glob import glob
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.decomposition import PCA
except:
    pass

try:
    from sklearn.preprocessing import normalize
except:
    pass

try:
    from keras.activations import softmax
except:
    pass

try:
    from keras.models import Sequential, load_model
except:
    pass

try:
    from keras.layers import Dense, Activation, Flatten, Dropout, Convolution2D, MaxPooling2D, AveragePooling2D
except:
    pass

try:
    from keras import backend as K
except:
    pass

try:
    from keras.optimizers import SGD
except:
    pass

try:
    from keras.utils import np_utils
except:
    pass

try:
    import sys
except:
    pass

try:
    from hyperas import opt

In [10]:
print("Evalutation of best performing model:")
print(best_model.evaluate(x_test, y_test))
print("Best performing model chosen hyper-parameters:")
print(best_run)
best_model.summary()

Evalutation of best performing model:
9470/9470 [==============================] - 1s 82us/step
[0.7017340095519012, 0.803590285047936]
Best performing model chosen hyper-parameters:
{'Activation': 0, 'Convolution2D': 1, 'Convolution2D_1': 1, 'Convolution2D_2': 1, 'Dense': 0, 'Dense_1': 2, 'Dropout': 0.06874772122693129, 'Dropout_1': 0.293803364162271, 'Dropout_2': 0.3746350041674067, 'Dropout_3': 0.2919157826643053, 'batch_size': 0, 'conditional': 1}
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 32, 95, 10)        672       
_________________________________________________________________
activation_51 (Activation)   (None, 32, 95, 10)        0         
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 32, 31, 5)         0         
_________________________________________________________________
dropout_33 (Drop

Da creare algoritmo che traduce il dizionario in rete